In [1]:
import pandas as pd
import numpy as np

In [2]:
df=pd.read_csv('cropRecommend_final.csv')
df.head()

,SO2,NO2,RSPM,SPM,N,P,K,temperature,humidity,ph,rainfall,label
0,9.140827,19.266625,85.913435,177.424864,90,42,43,20.879744,82.002744,6.502985,202.935536,rice
1,6.140827,22.266625,94.913435,165.424864,85,58,41,21.770462,80.319644,7.038096,226.655537,rice
2,11.140827,20.266625,91.913435,158.424864,60,55,44,23.004459,82.320763,7.840207,263.964248,rice
3,11.140827,18.266625,72.913435,169.424864,74,35,40,26.491096,80.158363,6.980401,242.864034,rice
4,5.140827,20.266625,75.913435,169.424864,78,42,42,20.130175,81.604873,7.628473,262.717340,rice


In [3]:
from sklearn import metrics
from sklearn import preprocessing
cat_col=['label',]
le=preprocessing.LabelEncoder()
for i in cat_col:
    le.fit(list(df[i].unique()))
    df[i]=le.transform(df[i])
x=df.iloc[:,0:11].values
y=df['label']
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.3,random_state=20)

In [4]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV

In [5]:
#create a search grid of parameters that will be shuffled through
param_grid={'bootstrap':[True],'max_depth':[10,20,30,40,50,60,70,80,90,100,None],
            'max_features':['auto','sqrt'],
            'min_samples_leaf':[1,2,4],'min_samples_split':[2,5,10],
            'n_estimators':[200,400,600,800,1000,1200]}
#Using the random grid and seaching for best hyperparameters
clf=RandomForestClassifier()
clf_random=RandomizedSearchCV(estimator=clf,param_distributions=param_grid,
                         n_iter=100,cv=5,verbose=2,n_jobs= -1,random_state=42)
clf_random.fit(x_train,y_train)

Fitting 5 folds for each of 100 candidates, totalling 500 fits


C:\ProgramData\anaconda3\Lib\site-packages\sklearn\model_selection\_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
C:\ProgramData\anaconda3\Lib\site-packages\sklearn\ensemble\_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


RandomizedSearchCV(cv=5, estimator=RandomForestClassifier(), n_iter=100,
                   n_jobs=-1,
                   param_distributions={'bootstrap': [True],
                                        'max_depth': [10, 20, 30, 40, 50, 60,
                                                      70, 80, 90, 100, None],
                                        'max_features': ['auto', 'sqrt'],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [200, 400, 600, 800,
                                                         1000, 1200]},
                   random_state=42, verbose=2)

In [6]:
clf_random.best_params_

{'n_estimators': 200,
 'min_samples_split': 2,
 'min_samples_leaf': 1,
 'max_features': 'auto',
 'max_depth': 80,
 'bootstrap': True}

In [7]:
def evaluate(model, x_test, y_test):
    predictions = model.predict(x_test)
    errors = abs(predictions - y_test)
    mape = 100 * np.mean(errors / y_test)
    accuracy = 100 - mape
    print('Model Performance')
    print('Average Error: {:0.4f} degrees.'.format(np.mean(errors)))
    print('Accuracy = {:0.2f}%.'.format(accuracy))
    
    return accuracy
base_model = RandomForestClassifier(n_estimators = 10, random_state = 42)
base_model.fit(x_train, y_train)
base_accuracy = evaluate(base_model, x_test, y_test)
best_random = clf_random.best_estimator_
random_accuracy = evaluate(best_random, x_test, y_test)

Model Performance
Average Error: 0.2228 degrees.
Accuracy = 97.30%.
Model Performance
Average Error: 0.1196 degrees.
Accuracy = 98.59%.
